In [2]:
from giskardpy.python_interface.python_interface import GiskardWrapper
import numpy as np
import pytest
import rospy
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, QuaternionStamped, Pose, Vector3
from nav_msgs.msg import Path
from numpy import pi
from shape_msgs.msg import SolidPrimitive
from tf.transformations import quaternion_from_matrix, quaternion_about_axis

import giskardpy.utils.tfwrapper as tf
from giskard_msgs.msg import WorldBody, CollisionEntry, WorldGoal, GiskardError
from giskardpy.configs.behavior_tree_config import StandAloneBTConfig
from giskardpy.configs.giskard import Giskard
from giskardpy.configs.iai_robots.pr2 import PR2CollisionAvoidance, PR2StandaloneInterface, WorldWithPR2Config
from giskardpy.configs.qp_controller_config import SupportedQPSolver, QPControllerConfig
from giskardpy.goals.base_traj_follower import FollowNavPath
from giskardpy.goals.cartesian_goals import RelativePositionSequence
from giskardpy.goals.caster import Circle, Wave
from giskardpy.goals.collision_avoidance import CollisionAvoidanceHint
from giskardpy.goals.goals_tests import DebugGoal, CannotResolveSymbol
from giskardpy.goals.joint_goals import JointVelocityLimit, UnlimitedJointGoal
from giskardpy.monitors.set_prediction_horizon import SetQPSolver
from giskardpy.goals.tracebot import InsertCylinder
from giskardpy.god_map import god_map
from giskardpy.model.better_pybullet_syncer import BetterPyBulletSyncer
from giskardpy.model.collision_world_syncer import CollisionWorldSynchronizer
from giskardpy.model.utils import make_world_body_box, hacky_urdf_parser_fix
from giskardpy.model.world import WorldTree
from giskardpy.data_types import PrefixName
from giskardpy.tasks.task import WEIGHT_BELOW_CA, WEIGHT_ABOVE_CA, WEIGHT_COLLISION_AVOIDANCE
from giskardpy.python_interface.old_python_interface import OldGiskardWrapper
from giskardpy.utils.utils import launch_launchfile, suppress_stderr, resolve_ros_iris
from giskardpy.utils.math import compare_points
from giskardpy.goals.weight_scaling_goals import MaxManipulabilityLinWeight, BaseArmWeightScaling
from copy import deepcopy

Set parameter ServerPassword
Set parameter ServerTimeout to value 10
Set parameter TokenServer to value "134.102.206.102"
[/unnamed]: Found these qp solvers: ['gurobi', 'qpalm', 'qpSWIFT']


In [3]:
rospy.init_node('giskard_examples')

In [4]:
gs = GiskardWrapper()

In [14]:
def reset_giskard():
    gs.clear_motion_goals_and_monitors()
    default_pose = {
        'r_elbow_flex_joint': -0.15,
        'r_forearm_roll_joint': 0,
        'r_shoulder_lift_joint': 0,
        'r_shoulder_pan_joint': 0,
        'r_upper_arm_roll_joint': 0,
        'r_wrist_flex_joint': -0.10001,
        'r_wrist_roll_joint': 0,
        'l_elbow_flex_joint': -0.15,
        'l_forearm_roll_joint': 0,
        'l_shoulder_lift_joint': 0,
        'l_shoulder_pan_joint': 0,
        'l_upper_arm_roll_joint': 0,
        'l_wrist_flex_joint': -0.10001,
        'l_wrist_roll_joint': 0,
        'torso_lift_joint': 0.2,
        'head_pan_joint': 0,
        'head_tilt_joint': 0,
        'l_gripper_l_finger_joint': 0.55,
        'r_gripper_l_finger_joint': 0.55
    }
    done = gs.monitors.add_set_seed_configuration(default_pose)
    base_pose = PoseStamped()
    base_pose.header.frame_id = 'map'
    base_pose.pose.position = Point(0, 0, 0)
    base_pose.pose.orientation.w = 1
    done2 = gs.monitors.add_set_seed_odometry(base_pose=base_pose)
    gs.motion_goals.allow_all_collisions()
    gs.monitors.add_end_motion(start_condition=done and done2)
    gs.execute()
    gs.clear_motion_goals_and_monitors()

Giskard is a constraint- and optimization-based whole-body motion planning/control framework.
In short explain the QP...
Explain a constraint and how it influences the QP...
When using the python wrapper you do not specify constraints directly but rather a Motion Goal to parameterize a predefined set of constraints.
Additionally, the python interface allows to specify monitors that monitor a mathematical expression against a threshold and evaluate into a binary value.
They are used to start, stop or interrupt motion goals. More complex motion goals might specify their own monitors to chain constraints together.
Show picture of PR2 TF Tree...

The simplest possible motion goal call follows the following schema:

    - Define a motion goal.
    - Define a monitor that checks if the goal was reached.
    - Define an end motion monitor, which has as start_condition the goal reached monitor.


In [8]:
reset_giskard()

joint_goal = {'torso_lift_joint': 0.3}
joint_monitor = gs.monitors.add_joint_position(goal_state=joint_goal)
gs.motion_goals.add_joint_position(goal_state=joint_goal)
gs.monitors.add_end_motion(start_condition=joint_monitor)
gs.execute()

error: 
  code: 0
  msg: ''
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_j

A commonly used goal is a cartesian pose goal. It is used to move a link of the robot to a specific pose. Frequently used links are the endeffector links or the base link.

In [9]:
reset_giskard()

# Define a goal pose in the map coordinate system
goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[1, 0, 0, 0],
                                                                 [0, 1, 0, 0],
                                                                 [0, 0, 1, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 2.01
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.7
# specify the cartesian pose goal to move 'l_gripper_tool_frame' towards goal_pose by using the kinematic chain between 'l_gripper_tool_frame' and 'map'
gs.motion_goals.add_cartesian_pose(goal_pose, 'l_gripper_tool_frame', 'map')
# specify monitor that is active when it is below the given thresholds and use it to end the motion
pose_monitor = gs.monitors.add_cartesian_pose('map', 'l_gripper_tool_frame', goal_pose, position_threshold=0.01, orientation_threshold=0.01)
gs.monitors.add_end_motion(pose_monitor)
# in the case that there is nos specific end motion condition the default end motion condition stops the motion when the robot stops moving i.e. the QP is in a local minimum
# gs.add_default_end_motion_conditions()
gs.execute()

error: 
  code: 0
  msg: ''
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_j

When adding an object to the world of giskard one can specify goal poses also in its coordinate frame. This is true for all links that are part of giskards world representation.

In [10]:
reset_giskard()

# adding a box to the world
box_pose = PoseStamped()
box_pose.header.frame_id = 'map'
box_pose.pose.orientation.w = 1
box_pose.pose.position.x = 2
box_pose.pose.position.y = -0.2
box_pose.pose.position.z = 0.7
gs.world.add_box('mybox', size=(0.1, 0.05, 0.2), pose=box_pose, parent_link='map')

# specfy a goal pose 8cm in front of the box origin frame
goal_pose = PoseStamped()
goal_pose.header.frame_id = 'mybox'
goal_pose.pose.orientation.w = 1
goal_pose.pose.position.x = -0.08
goal_pose.pose.position.y = 0
goal_pose.pose.position.z = 0
gs.motion_goals.add_cartesian_pose(goal_pose, 'l_gripper_tool_frame', 'map')
pose_monitor = gs.monitors.add_cartesian_pose('map', 'l_gripper_tool_frame', goal_pose, position_threshold=0.01, orientation_threshold=0.01)
gs.monitors.add_end_motion(pose_monitor)
gs.execute()

error: 
  code: 0
  msg: ''
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_j

When using the cartesian goals all degrees of freedom of the link are fully constrained. One Advantage of constraint-based control is that one can only constrain neccessary constraints.
The unconstrained DOF can then be exploited during the motion, for example in the collision avoidance or other secondary tasks.
To use this in the python interface one can use motion goals that constrain only a singualar value (sometimes called constraints on feature functions).|

Combination of constraints and cost function augmentations to
- place both grippers at specific poses (2 6D Pose Constraints)
- keep the line of sight on the left gripper (Pointing Constraint)
- prefer using base movements if the grippers are further away from their goal poses and avoid base movements if they are close to their goals (Quadratic Weight Scaling)
- maximize the manipulability in each arm (Linear weight term in the cost function)

In [13]:
# reset_giskard()

js = {
    # 'torso_lift_joint': 0.2999225173357618,
    'head_pan_joint': 0.041880780651479044,
    'head_tilt_joint': -0.37,
    'r_upper_arm_roll_joint': -0.9487714747527726,
    'r_shoulder_pan_joint': -1.0047307505973626,
    'r_shoulder_lift_joint': 0.48736790658811985,
    'r_forearm_roll_joint': -14.895833882874182,
    'r_elbow_flex_joint': -1.392377908925028,
    'r_wrist_flex_joint': -0.4548695149411013,
    'r_wrist_roll_joint': 0.11426798984097819,
    'l_upper_arm_roll_joint': 1.7383062350263658,
    'l_shoulder_pan_joint': 1.8799810286792007,
    'l_shoulder_lift_joint': 0.011627231224188975,
    'l_forearm_roll_joint': 312.67276414458695,
    'l_elbow_flex_joint': -2.0300928925694675,
    'l_wrist_flex_joint': -0.10014623223021513,
    'l_wrist_roll_joint': -6.062015047706399,
}
gs.motion_goals.add_joint_position(js)
gs.motion_goals.allow_all_collisions()
gs.add_default_end_motion_conditions()
gs.execute()

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[1, 0, 0, 0],
                                                                 [0, 1, 0, 0],
                                                                 [0, 0, 1, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 2.01
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.7

goal_pose2 = deepcopy(goal_pose)
goal_pose2.pose.position.y = -0.6
goal_pose2.pose.position.z = 0.8
goal_pose2.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, 1, 0],
                                                                  [0, 1, 0, 0],
                                                                  [-1, 0, 0, 0],
                                                                  [0, 0, 0, 1]]))

gs.motion_goals.add_cartesian_pose(goal_pose, 'l_gripper_tool_frame', 'map')
gs.motion_goals.add_cartesian_pose(goal_pose2, 'r_gripper_tool_frame', 'map', name='cart2')

goal_point = PointStamped()
goal_point.header.frame_id = goal_pose.header.frame_id
goal_point.point = goal_pose.pose.position
pointing_axis = Vector3Stamped()
pointing_axis.header.frame_id = 'wide_stereo_optical_frame'
pointing_axis.vector.z = 1
gs.motion_goals.add_pointing(goal_point, 'wide_stereo_optical_frame', pointing_axis, 'map')

x_base = Vector3Stamped()
x_base.header.frame_id = 'base_link'
x_base.vector.x = 1
x_goal = Vector3Stamped()
x_goal.header.frame_id = 'map'
x_goal.vector.x = 1
gs.motion_goals.add_align_planes(tip_link='base_link',
                                 root_link='map',
                                 tip_normal=x_base,
                                 goal_normal=x_goal)

tip_goal = PointStamped()
tip_goal.header.frame_id = 'map'
tip_goal.point = goal_pose.pose.position
gs.motion_goals.add_motion_goal(motion_goal_class=BaseArmWeightScaling.__name__,
                                root_link='map',
                                tip_link='l_gripper_tool_frame',
                                tip_goal=tip_goal,
                                gain=100000,
                                arm_joints=[
                                    'torso_lift_joint',
                                    # 'head_pan_joint',
                                    # 'head_tilt_joint',
                                    'r_upper_arm_roll_joint',
                                    'r_shoulder_pan_joint',
                                    'r_shoulder_lift_joint',
                                    'r_forearm_roll_joint',
                                    'r_elbow_flex_joint',
                                    'r_wrist_flex_joint',
                                    'r_wrist_roll_joint',
                                    'l_upper_arm_roll_joint',
                                    'l_shoulder_pan_joint',
                                    'l_shoulder_lift_joint',
                                    'l_forearm_roll_joint',
                                    'l_elbow_flex_joint',
                                    'l_wrist_flex_joint',
                                    'l_wrist_roll_joint'],
                                base_joints=['brumbrum'])
gs.motion_goals.add_motion_goal(motion_goal_class=MaxManipulabilityLinWeight.__name__,
                                root_link='torso_lift_link',
                                tip_link='r_gripper_tool_frame')
gs.motion_goals.add_motion_goal(motion_goal_class=MaxManipulabilityLinWeight.__name__,
                                root_link='torso_lift_link',
                                tip_link='l_gripper_tool_frame',
                                name='MaxMal2')
gs.add_default_end_motion_conditions()
gs.motion_goals.allow_all_collisions()
gs.execute()

error: 
  code: 0
  msg: ''
trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  joint_names: 
    - fl_caster_rotation_joint
    - fl_caster_l_wheel_joint
    - fl_caster_r_wheel_joint
    - fr_caster_rotation_joint
    - fr_caster_l_wheel_joint
    - fr_caster_r_wheel_joint
    - bl_caster_rotation_joint
    - bl_caster_l_wheel_joint
    - bl_caster_r_wheel_joint
    - br_caster_rotation_joint
    - br_caster_l_wheel_joint
    - br_caster_r_wheel_joint
    - torso_lift_joint
    - head_pan_joint
    - head_tilt_joint
    - laser_tilt_mount_joint
    - r_shoulder_pan_joint
    - r_shoulder_lift_joint
    - r_upper_arm_roll_joint
    - r_elbow_flex_joint
    - r_forearm_roll_joint
    - r_wrist_flex_joint
    - r_wrist_roll_joint
    - r_gripper_motor_slider_joint
    - r_gripper_motor_screw_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_l_finger_joint
    - r_gripper_j